In [1]:
! pip install -q tensorflow-model-optimization

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import tensorflow as tf

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2159161636853275263
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7810842624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14957317362282087821
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:06:00.0, compute capability: 8.6"
]


In [3]:
import tensorflow as tf
import argparse
import importlib  
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import os

from tensorflow.keras.preprocessing import image

from utils.keras_functions import sparse_crossentropy_ignoring_last_label, Jaccard

keras_deeplab = importlib.import_module("keras-deeplab-v3-plus.model")

import Keras_segmentation_deeplab_v3_1.utils

monitor = 'Jaccard'
mode = 'max'

losses = sparse_crossentropy_ignoring_last_label
metrics = {'pred_mask' : [Jaccard]}

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Version 2.x.x


In [4]:
def build_model(model_name, os, alpha, norm):
    try:
        deeplab_model = keras_deeplab.Deeplabv3(backbone=model_name, input_shape=(375, 513, 3), classes=19, weights='cityscapes', OS=os, alpha=alpha, infer=False, normalization=norm)
    except:
        raise Exception("No model with given backbone: ", model_name)

    deeplab_model.compile(optimizer = Adam(lr=7e-4, epsilon=1e-8, decay=1e-6), sample_weight_mode = "temporal",
              loss = losses, metrics = metrics)
    return deeplab_model

def load_model(path):
    deeplab_model = keras_deeplab.Deeplabv3(backbone='xception', input_shape=(375, 513, 3), classes=19, weights='cityscapes', OS=16, alpha=1, infer=False, normalization=0)
    deeplab_model.load_weights(path)

    return deeplab_model

In [5]:
model = load_model('/mnt/7BCDA59C6DEFFE3C/kitti_weights/xception_retrained_375_0001_rndcrop_infinite/xception_retrained_375_0001_rndcrop_infinite_model.21-0.27-0.53.h5')

Cityscapes!
166100992/166094648 [==============================] - 9s 0us/step


In [7]:
model.summary()

Model: "deeplabv3plus"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 375, 513, 3) 0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 375, 513, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 188, 257, 32) 864         lambda_6[0][0]                   
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 188, 257, 32) 128         entry_flow_conv1_1[0][0]         
______________________________________________________________________________________

In [8]:
import tensorflow_model_optimization as tfmot
from keras.optimizers import Adam
import numpy as np

end_step = np.ceil(train_generator.true_len() / 4).astype(np.int32) * 4

def apply_pruning_to_dense(layer):
    #if isinstance(layer, tf.keras.layers.DepthwiseConv2D) 
    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
    }
    if isinstance(layer, tf.keras.layers.DepthwiseConv2D): #or isinstance(layer, tf.keras.layers.Conv2D):
        return tfmot.sparsity.keras.prune_low_magnitude(layer)
    return layer

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense` 
# to the layers of the model.
model_for_pruning = tf.keras.models.clone_model(
    model,
    clone_function=apply_pruning_to_dense,
)

model_for_pruning.compile(optimizer=Adam(lr=0.0001, epsilon=1e-8, decay=1e-6),
              loss=losses,
              metrics=metrics)

model_for_pruning.summary()

Model: "deeplabv3plus"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 375, 513, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 375, 513, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 188, 257, 32) 864         lambda[1][0]                     
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 188, 257, 32) 128         entry_flow_conv1_1[1][0]         
______________________________________________________________________________________

In [7]:
SegClass = Keras_segmentation_deeplab_v3_1.utils.SegModel('../kitti360_dataset.csv', image_size=(375,513))
SegClass.set_num_epochs(4)
SegClass.set_batch_size(4)


train_generator = SegClass.create_generators(dataset = '../kitti360_dataset.csv', blur=0, mode='train',
                                                n_classes=19, horizontal_flip=False, vertical_flip=False, 
                                                brightness=0, rotation=False, zoom=0, batch_size=4,
                                                seed=7, do_ahisteq=False, random_crop=True, resize_shape=(375,513))

valid_generator = SegClass.create_generators(dataset ='../kitti360_dataset.csv', blur=0, mode='val',
                                                n_classes=19, horizontal_flip=False, vertical_flip=False, 
                                                brightness=0, rotation=False, zoom=0, batch_size=4,
                                                seed=7, do_ahisteq=False, resize_shape=(375,513))

In [9]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='pruning_log_v5'),
]

model_for_pruning.fit(train_generator,
                  batch_size=4, epochs=4, validation_data=valid_generator,
                  callbacks=callbacks)

Epoch 1/4
Instructions for updating:
Use `tf.cast` instead.
10855/10855 [==============================] - 3633s 333ms/step - loss: 0.4649 - Jaccard: 0.4782 - val_loss: 0.7852 - val_Jaccard: 0.4479
End of epoch...
Epoch 2/4
10855/10855 [==============================] - 3800s 350ms/step - loss: 0.3723 - Jaccard: 0.4464 - val_loss: 0.3968 - val_Jaccard: 0.4413
End of epoch...
Epoch 3/4
10855/10855 [==============================] - 3684s 339ms/step - loss: 0.3741 - Jaccard: 0.4453 - val_loss: 0.4174 - val_Jaccard: 0.4381
End of epoch...
Epoch 4/4
10855/10855 [==============================] - 3799s 350ms/step - loss: 0.3773 - Jaccard: 0.4433 - val_loss: 0.4150 - val_Jaccard: 0.4343
End of epoch...


In [10]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

tf.keras.models.save_model(model_for_export, '../pruned_models/depth_wise.h5', include_optimizer=False)
print('Saved pruned Keras model')

Saved pruned Keras model
